# Ghost Attack ~ all frequencies: BJH026

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
import os
import mne
import IPython
import seaborn as sns
import scipy
import joblib
import h5io
import dask.array as da 

import statsmodels
from statsmodels import stats
from statsmodels.stats import multitest

# Import required code for visualizing example models
from fooof import FOOOF
from fooof.sim.gen import gen_power_spectrum
from fooof.sim.utils import set_random_seed
from fooof.plts.spectra import plot_spectra
from fooof.plts.annotate import plot_annotated_model
from neurodsp.utils import create_times
from neurodsp.plts.time_series import plot_time_series
from neurodsp.spectral import compute_spectrum, rotate_powerlaw
from neurodsp.plts.spectral import plot_power_spectra


In [ ]:
## Prep paths ##

subject = 'BJH026'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

# load preproc functions
%run ../../scripts/preproc_functions.py

In [ ]:
## Load Neural Data

# load
ghost_attack_epochs = mne.read_epochs(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif")

# get good epochs (for behavioral data only)
good_epochs = [i for i,x in enumerate(ghost_attack_epochs.get_annotations_per_epoch()) if not x]
bad_epochs = [i for i,x in enumerate(ghost_attack_epochs.get_annotations_per_epoch()) if  x]

# load behavioral data
ghost_attack_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_attack_events.csv")

# set info as metadata
ghost_attack_epochs.metadata = ghost_attack_data

# onlt good epochs
ghost_attack_epochs = ghost_attack_epochs[good_epochs]

In [ ]:
## Dictionary of electrode locations ##

# Pull mapping ROI to elecs
%run ../../scripts/roi.py
ROIs = ROIs[subject]

## prep lists

# limbic ROI
hc_list = []
hc_indices = []
hc_names = []
ofc_list = []
ofc_indices = []
ofc_names = []
amyg_list = []
amyg_names = [] 
amyg_indices = []
cing_list = []
cing_names = [] 
cing_indices = []
sgACC_list = []
sgACC_names = [] 
sgACC_indices = []
dACC_list = []
dACC_names = [] 
dACC_indices = []
sfg_list = []
sfg_names = [] 
sfg_indices = []
mfg_list = []
mfg_names = [] 
mfg_indices = []

# control ROI
insula_list = []
insula_names = []  
insula_indices = []
dlpfc_list = []
dlpfc_names = []  
dlpfc_indices = []
ec_list = []
ec_names = []  
ec_indices = []

# exclude bad ROI from list
pairs_long_name = [ch.split('-') for ch in ghost_attack_epochs.info['ch_names']]
bidx = len(ghost_attack_epochs.info['bads']) +1
pairs_name = pairs_long_name[bidx:len(pairs_long_name)]

# sort ROI into lists
for ix in range(0, len(pairs_name)):
    if pairs_name[ix][0] in ROIs['hc'] or pairs_name[ix][1] in ROIs['hc']:
        hc_list.append(ghost_attack_epochs.info['ch_names'][ix + bidx])
        hc_names.append(pairs_name[ix])
        hc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['ofc'] or pairs_name[ix][1] in ROIs['ofc']:
        ofc_list.append(ghost_attack_epochs.info['ch_names'][ix + bidx])
        ofc_names.append(pairs_name[ix])
        ofc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['amyg'] or pairs_name[ix][1] in ROIs['amyg']:
        amyg_list.append(ghost_attack_epochs.info['ch_names'][ix + bidx])       
        amyg_names.append(pairs_name[ix])
        amyg_indices.append(ix)
    if pairs_name[ix][0] in ROIs['cing'] or pairs_name[ix][1] in ROIs['cing']:
        cing_list.append(ghost_attack_epochs.info['ch_names'][ix + bidx])       
        cing_names.append(pairs_name[ix])
        cing_indices.append(ix)
    if pairs_name[ix][0] in ROIs['sgACC'] or pairs_name[ix][1] in ROIs['sgACC']:
        sgACC_list.append(ghost_attack_epochs.info['ch_names'][ix + bidx])       
        sgACC_names.append(pairs_name[ix])
        sgACC_indices.append(ix)
    if pairs_name[ix][0] in ROIs['dACC'] or pairs_name[ix][1] in ROIs['dACC']:
        dACC_list.append(ghost_attack_epochs.info['ch_names'][ix + bidx])       
        dACC_names.append(pairs_name[ix])
        dACC_indices.append(ix)        
        
    # control roi
    if pairs_name[ix][0] in ROIs['insula'] or pairs_name[ix][1] in ROIs['insula']:
        insula_list.append(ghost_attack_epochs.info['ch_names'][ix + bidx])       
        insula_names.append(pairs_name[ix])
        insula_indices.append(ix)
    if pairs_name[ix][0] in ROIs['dlpfc'] or pairs_name[ix][1] in ROIs['dlpfc']:
        dlpfc_list.append(ghost_attack_epochs.info['ch_names'][ix + bidx])       
        dlpfc_names.append(pairs_name[ix])
        dlpfc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['sfg'] or pairs_name[ix][1] in ROIs['sfg']:
        sfg_list.append(ghost_attack_epochs.info['ch_names'][ix + bidx])       
        sfg_names.append(pairs_name[ix])
        sfg_indices.append(ix)   
    if pairs_name[ix][0] in ROIs['mfg'] or pairs_name[ix][1] in ROIs['mfg']:
        mfg_list.append(ghost_attack_epochs.info['ch_names'][ix + bidx])       
        mfg_names.append(pairs_name[ix])
        mfg_indices.append(ix)     
    if pairs_name[ix][0] in ROIs['ec'] or pairs_name[ix][1] in ROIs['ec']:
        ec_list.append(ghost_attack_epochs.info['ch_names'][ix + bidx])       
        ec_names.append(pairs_name[ix])
        ec_indices.append(ix)        
        

In [ ]:
# Print and check frequencies
print(freqs)
print(n_cycles)
print(time_bin)
print(band_width)


# Main Regions of Interest

# OFC

In [ ]:
ofc_list

In [ ]:
# only roi of interest
ghost_attack_roi = ghost_attack_epochs.copy().pick_channels(ofc_list)


In [ ]:
# Resample to 1000 
if ghost_attack_roi.info['sfreq'] > 1000:
    ghost_attack_roi= ghost_attack_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = compute_TFR(ghost_attack_roi,freqs,n_cycles)

In [ ]:
roi_tfr.crop(tmin = -1, tmax = 3)


In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/ghost_attack/ofc-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = log_and_zscore_TFR(roi_tfr, baseline = (-1,3), logflag=True)

In [ ]:
# Plot TFR
plot_average_tfr(roi_tfr, f"Average OFC TFR at Ghost Attack, n={len(ofc_list)}", subject, 'ofc_ghostattack')


### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    plot_channel_tfr(roi_tfr, chix, ch, 'Ghost Attack')

# Amygdala

In [ ]:
amyg_list

In [ ]:
# only roi 
ghost_attack_roi = ghost_attack_epochs.copy().pick_channels(amyg_list)


In [ ]:
# Resample to 1000 
if ghost_attack_roi.info['sfreq'] > 1000:
    ghost_attack_roi= ghost_attack_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = compute_TFR(ghost_attack_roi,freqs,n_cycles)


In [ ]:
roi_tfr.crop(tmin = -1, tmax = 3)


In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/ghost_attack/amyg-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = log_and_zscore_TFR(roi_tfr, baseline = (-1,3), logflag=True)

In [ ]:
# Plot TFR
plot_average_tfr(roi_tfr, f"Average Amygdala TFR at Ghost Attack, n={len(amyg_list)}", subject, 'amyg_ghostattack')

### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    plot_channel_tfr(roi_tfr, chix, ch, 'Ghost Attack')

## Cingulate

In [ ]:
cing_list

In [ ]:
# only roi 
ghost_attack_roi = ghost_attack_epochs.copy().pick_channels(cing_list)


In [ ]:
# Resample to 1000 
if ghost_attack_roi.info['sfreq'] > 1000:
    ghost_attack_roi= ghost_attack_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = compute_TFR(ghost_attack_roi,freqs,n_cycles)


In [ ]:
roi_tfr.crop(tmin = -1, tmax = 3)


In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/ghost_attack/cing-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = log_and_zscore_TFR(roi_tfr, baseline = (-1,3), logflag=True)

In [ ]:
# Plot TFR
plot_average_tfr(roi_tfr, f"Average Ant. Cingulate TFR at Ghost Attack, n={len(cing_list)}", subject, 'cing_ghostattack')

### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    plot_channel_tfr(roi_tfr, chix, ch, 'Ghost Attack')

# Control Regions

## Insula

In [ ]:
insula_list

In [ ]:
# Only ROI
ghost_attack_roi = ghost_attack_epochs.copy().pick_channels(insula_list)


In [ ]:
# Resample to 1000 
if ghost_attack_roi.info['sfreq'] > 1000:
    ghost_attack_roi= ghost_attack_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = compute_TFR(ghost_attack_roi,freqs,n_cycles)


In [ ]:
roi_tfr.crop(tmin = -1, tmax = 3)


In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/ghost_attack/insula-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = log_and_zscore_TFR(roi_tfr, baseline = (-1,3), logflag=True)

In [ ]:
# Plot TFR
plot_average_tfr(roi_tfr, f"Average Insula TFR at Ghost Attack, n={len(insula_list)}", subject, 'insula_ghostattack')

### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    plot_channel_tfr(roi_tfr, chix, ch, 'Ghost Attack')

## dlPFC

In [ ]:
dlpfc_list

In [ ]:
# Only ROI
ghost_attack_roi = ghost_attack_epochs.copy().pick_channels(dlpfc_list)


In [ ]:
# Resample to 1000 
if ghost_attack_roi.info['sfreq'] > 1000:
    ghost_attack_roi= ghost_attack_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = compute_TFR(ghost_attack_roi,freqs,n_cycles)

In [ ]:
roi_tfr.crop(tmin = -1, tmax = 3)


In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/ghost_attack/dlpfc-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = log_and_zscore_TFR(roi_tfr, baseline = (-1,3), logflag=True)

In [ ]:
# Plot TFR
plot_average_tfr(roi_tfr, f"Average dlPFC TFR at Ghost Attack, n={len(dlpfc_list)}", subject, 'dlpfc_ghostattack')

### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    plot_channel_tfr(roi_tfr, chix, ch, 'Ghost Attack')